In [25]:
import tensorflow as tf
import numpy as np
import gym
from go_ai import data, metrics, policies
from go_ai.models import value_model
import matplotlib.pyplot as plt
import shutil
import multiprocessing as mp

# Hyperparameters

In [26]:
BOARD_SIZE = 9

In [27]:
ITERATIONS = 0
EPISODES_PER_ITERATION = 128
NUM_EVAL_GAMES = 32

In [28]:
EPISODES_DIR = './data/'

In [29]:
BATCH_SIZE = 32
LEARNING_RATE = 2e-3

In [30]:
WEIGHTS_DIR = 'model_weights/'
CHECKPOINT_PATH = WEIGHTS_DIR + 'checkpoint_{}x{}.h5'.format(BOARD_SIZE, BOARD_SIZE)
TMP_WEIGHTS_PATH = WEIGHTS_DIR + 'tmp.h5'
LOAD_SAVED_MODELS = False

In [31]:
NUM_WORKERS = mp.cpu_count()

# Go Environment
Train on a small board for fast training and efficient debugging

In [32]:
go_env = gym.make('gym_go:go-v0', size=BOARD_SIZE)

# Metrics and Tensorboard

In [33]:
DEMO_TRAJECTORY_PATH = 'logs/a_trajectory.png'

Metrics

In [34]:
tb_metrics = {}
for metric_key in ['val_loss', 'move_loss']:
    tb_metrics[metric_key] = tf.keras.metrics.Mean('{}'.format(metric_key), 
                                                   dtype=tf.float32)
tb_metrics['pred_win_acc'] = tf.keras.metrics.Accuracy()

Tensorboard

# Machine Learning Models

In [35]:
val_net = value_model.make_val_net(BOARD_SIZE)

In [36]:
_ = tf.keras.utils.plot_model(val_net, to_file='logs/model.png')

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


In [37]:
val_net.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 7, 7, 128)         7040      
_________________________________________________________________
batch_normalization_16 (Batc (None, 7, 7, 128)         512       
_________________________________________________________________
re_lu_16 (ReLU)              (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 7, 7, 128)         147584    
_________________________________________________________________
batch_normalization_17 (Batc (None, 7, 7, 128)         512       
_________________________________________________________________
re_lu_17 (ReLU)              (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 7, 7, 64)         

In [38]:
if LOAD_SAVED_MODELS:
    val_net.load_weights(CHECKPOINT_PATH)
    print("Starting from checkpoint")
else:
    val_net.save_weights(CHECKPOINT_PATH)
    print("Initialized checkpoint and temp")
    
# Sync temp with checkpoint
val_net.save_weights(TMP_WEIGHTS_PATH)

OSError: Unable to open file (unable to open file: name = 'model_weights/checkpoint_7x7.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

# Policies

In [15]:
temp_policy_args = {
    'mode': 'values',
    'board_size': BOARD_SIZE,
    'model_path': TMP_WEIGHTS_PATH,
}

In [16]:
checkpoint_policy_args = {
    'mode': 'values',
    'board_size': BOARD_SIZE,
    'model_path': CHECKPOINT_PATH,
}

In [17]:
random_policy_args = {
    'mode': 'random',
    'board_size': BOARD_SIZE,
}

In [18]:
greedy_policy_args = {
    'mode': 'greedy',
    'board_size': BOARD_SIZE,
}

# Demo and Time Games

Symmetries

In [19]:
%%time
go_env.reset()
action = (1, 2)
next_state, _, _, _ = go_env.step(action)
metrics.plot_symmetries(next_state, 'logs/symmetries.jpg')

CPU times: user 133 ms, sys: 8.7 ms, total: 142 ms
Wall time: 140 ms


Without replay memory

In [20]:
%%time
data.make_episodes(temp_policy_args, temp_policy_args, 1, num_workers=1)

Episode worker: 0it [00:00, ?it/s]

KeyboardInterrupt: 

With replay memory

In [21]:
%%time
data.make_episodes(temp_policy_args, temp_policy_args, 1, num_workers=1, 
                   outdir=EPISODES_DIR)

Episode worker: 1it [00:23, 23.38s/it]

values vs. values: 100%|██████████| 1/1 [00:00<00:00, 636.46it/s, 100.0% WIN]

CPU times: user 28.2 s, sys: 2.34 s, total: 30.6 s
Wall time: 23.6 s


1.0

# Train

In [22]:
for iteration in range(ITERATIONS):
    # Optimization
    
    # Make and write out the episode data
    data.make_episodes(temp_policy_args, temp_policy_args, EPISODES_PER_ITERATION, 
                       num_workers=NUM_WORKERS, outdir=EPISODES_DIR)
    # Read in the episode data
    np_data = data.episodes_from_dir(EPISODES_DIR)
    batched_np_data = [np.array_split(datum, len(np_data[0]) // BATCH_SIZE) for datum in np_data]
    batched_mem = list(zip(*batched_np_data))

    # Optimize
    value_model.optimize_val_net(temp_policy_args, batched_mem, LEARNING_RATE, tb_metrics)
    # Resets the metrics
    metrics.reset_metrics(tb_metrics)
    
    # Evaluate against checkpoint model and other baselines
    rand_win_rate = data.make_episodes(temp_policy_args, random_policy_args, 
                                       NUM_EVAL_GAMES, num_workers=NUM_WORKERS)
    opp_win_rate = data.make_episodes(temp_policy_args, checkpoint_policy_args, 
                                      NUM_EVAL_GAMES, num_workers=NUM_WORKERS)

    stats = f"{100*opp_win_rate:.1f}%O, {100*rand_win_rate:.1f}%R"

    # If it's better than the checkpoint, update
    if opp_win_rate > 0.6:
        shutil.copy(TMP_WEIGHTS_PATH, CHECKPOINT_PATH)
        print(f"{stats} Accepted new model")
        
        greed_win_rate = data.make_episodes(temp_policy_args, greedy_policy_args, 
                                        NUM_EVAL_GAMES, num_workers=NUM_WORKERS)

        # Plot samples of states and response heatmaps
        fig = metrics.gen_traj_fig(go_env, temp_policy_args)
        fig.savefig(DEMO_TRAJECTORY_PATH)
        plt.close()

    elif opp_win_rate >= 0.5:
        print(f"{stats} Continuing to train current weights")

    else:
        shutil.copy(CHECKPOINT_PATH, TMP_WEIGHTS_PATH)
        print(f"{stats} Rejected new model")

# Evaluate

Play against our AI

In [23]:
human_policy_args = {
    'mode': 'human',
    'board_size': BOARD_SIZE,
}

In [24]:
data.make_episodes(checkpoint_policy_args, human_policy_args, 1, num_workers=1)

Episode worker: 0it [00:00, ?it/s]

    0   1   2   3   4   5   6   7   8
  -------------------------------------
0 | . | . | . | . | . | . | . | . | . |
  -------------------------------------
1 | . | . | . | . | . | . | . | . | . |
  -------------------------------------
2 | . | . | . | . | . | . | . | . | . |
  -------------------------------------
3 | . | . | . | . | . | . | . | . | . |
  -------------------------------------
4 | . | . | . | . | . | . | . | . | . |
  -------------------------------------
5 | . | . | . | . | . | . | . | . | . |
  -------------------------------------
6 | . | . | . | . | . | . | . | . | . |
  -------------------------------------
7 | . | . | . | . | . | . | . | B | . |
  -------------------------------------
8 | . | . | . | . | . | . | . | . | . |
  -------------------------------------
	Turn: W, Last Turn Passed: False, Game Over: 0
	Black Area: 81, White Area: 0

Enter coordinates separated by space (`q` to quit)
q
invalid literal for int() with base 10: 'q'
    0   1   2   3   4   5

Episode worker: 1it [00:47, 47.29s/it]

values vs. human: 100%|██████████| 1/1 [00:00<00:00, 500.27it/s, 100.0% WIN]


1.0